### Code to run web Interface ###

In [ ]:
#Import require Flask library
from flask import Flask,redirect,url_for, render_template, request, session, Response
from webcam import Camera
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import desc
from datetime import datetime
from random import randrange


#creating teflask app
app1 = Flask(__name__)
app1.secret_key = "HELLO"
#configuring SQLALCHEMY_DATABASE
app1.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///users.sqlite3'
app1.config["SQLALCHEMY_TRACK_MODIFICATIONS"] = False

label1 = 0
people_in = 0

#creating database for app
db = SQLAlchemy(app1)

#configuring visitors table in the database 
class visitors(db.Model):
    _id = db.Column("id", db.Integer, primary_key=True) #unique id number
    operation = db.Column(db.String(100)) #opration type
    value = db.Column(db.Integer)  #visitor count value
    total_count = db.Column(db.Integer)  #Total count of visitor
    timestamp = db.Column(db.DateTime)  #timestamp on each entry
    
    def __init__(self, _id, operation, value, total_count, timestamp):
        self._id = _id
        self.operation = operation
        self.value = value
        self.total_count = total_count
        self.timestamp = timestamp
    
#Function to define override mode
def over(value):
    session["id"] += 100
    #multiple override
    if value == "Multiple":
        session["override"] = 2
        rand_val = people_in
        session["visitor"] += rand_val
        #add visitor
        visit = visitors(session["id"],"Manual Override", rand_val, session["visitor"], datetime.now())
    #single override
    elif value == "Single":
        session["override"] = 1
        session["visitor"] += 1
        #add visitor
        visit = visitors(session["id"],"Single Override", 1, session["visitor"], datetime.now())
    #adding data in database
    db.session.add(visit)
    #saving the transaction 
    db.session.commit()

#Flask module for home page
@app1.route("/", methods=["POST","GET"])
def home():
    
    
    global label1
    
    #perform actions based on type of request
    if request.method == "POST":
        val = request.form["do"]
        if val == "Cancel":
              session["override"] = 0
        else:
            over(val)
        return redirect(url_for("home"))
    else:
        if label1 == 0:
            session["visitor"] = 0
            session["override"] = 0
            session["id"] = 0
            label1 = 1
        return render_template("home.html", values= session["visitor"])
        
#function to get webcam input
def gen(camera):
    while True:
        
        frame = camera.get_frame()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
    
#Flask module for video_feed 
@app1.route("/video_feed")
def video_feed():
    
    return Response(gen(Camera()),
                    mimetype='multipart/x-mixed-replace; boundary=frame')
    
    
    

#Flask module for visitor page 
@app1.route("/visitor", methods=["POST","GET"])
def visitor():
    
    global people_in
    
    if(people_in != 0):
        
        session["id"] += 5
        session["visitor"] += people_in
        #add ardunio input to the database
        visit = visitors(session["id"],"Visitor_Page", people_in, session["visitor"], datetime.now())
        db.session.add(visit)
        db.session.commit()
        people_in = 0
        
    
    if request.method == "POST":

        val = request.form["do"]
        vis(val)
        return redirect(url_for("visitor"))
    else:
        return render_template("visitor.html", values = visitors.query.order_by(desc(visitors.timestamp)).all())

        
    
if __name__ == "__main__":
    label1 = 0
    people_in = 0
    db.drop_all()
    db.create_all()
    app1.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Oct/2020 12:49:31] "GET /visitor HTTP/1.1" 200 -
127.0.0.1 - - [30/Oct/2020 12:49:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Oct/2020 12:49:51] "GET /video_feed HTTP/1.1" 200 -
127.0.0.1 - - [30/Oct/2020 12:49:54] "GET /visitor HTTP/1.1" 200 -
127.0.0.1 - - [30/Oct/2020 12:49:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Oct/2020 12:49:58] "GET /video_feed HTTP/1.1" 200 -
